In [ ]:
import copy
from pathlib import Path
import random
from time import sleep, time

import anthropic
from anthropic.types.message_create_params import MessageCreateParamsNonStreaming
from anthropic.types.messages.batch_create_params import Request
from claudette import mk_msg

In [ ]:
relevant_comment_example1 =\
"""
Function Definition:
'''
def _create_rearrange_callable(
    tensor_ndim: int, pattern: str, **axes_lengths: int
) -> Callable[[torch.Tensor], torch.Tensor]
'''

Code:
'''
    n_dims = n_named_dims + n_ellipsis_dims + n_anon_dims
    
    if n_dims == 0:
        # an identity rearrangement on a 0-dimension tensor
        return lambda tensor: tensor
    
    first_class_dims: Tuple[str, ...] = tuple(f"d{i}" for i in range(n_dims))
'''

Comment:
'''
# an identity rearrangement on a 0-dimension tensor
'''

Explanation:
'''
The comment indicates that for a 0-dimension tensor we have an early return.
'''

Correct:
true
"""

relevant_comment_example2 =\
"""
Function Definition:
'''
def setup_loading_other_datasets()
'''

Code:
'''
    except ImportError:
        raise SkipTest("Skipping loading_other_datasets.rst, pandas not installed")
    
    # checks SKLEARN_SKIP_NETWORK_TESTS to see if test should run
    run_network_tests = environ.get("SKLEARN_SKIP_NETWORK_TESTS", "1") == "0"
    if not run_network_tests:
        raise SkipTest(
'''

Comment:
'''
# checks SKLEARN_SKIP_NETWORK_TESTS to see if test should run
'''

Explanation:
'''
While the code might be considered a bit redundant, it does correctly explain what is being done and for what purpose.
'''

Correct:
true
"""

irrelevant_comment_example1 =\
"""
Function Definition:
'''
def __setstate__(self, state) -> None:
'''

Code:
'''
    if state[0] is None:
        # create a reference from the input state
        self.hooks_dict_ref = weakref.ref(OrderedDict())
    else:
        self.hooks_dict_ref = weakref.ref(state[0])
    self.id = state[1]
'''

Comment:
'''
# create a reference from the input state
'''

Explanation:
'''
While the comment does talk about creating a reference, the reference is not created from the input state.
'''

Correct:
false
"""

irrelevant_comment_example2 =\
"""
Function Definition:
'''
def sin(
    angle_in_degrees: float, accuracy: int = 18, rounded_values_count: int = 10
) -> float
'''

Code:
'''
    # Simplify the angle to be between 360 and -360 degrees.
    angle_in_degrees = angle_in_degrees - ((angle_in_degrees // 360.0) * 360.0)

    # Converting from degrees to radians
    angle_in_radians = radians(angle_in_degrees)

    result = angle_in_radians
    a = 3
    b = -1
'''

Comment:
'''
# Converting from degrees to radians
'''

Explanation:
'''
While the comment is correct, it's completely redundant given that it basically rewords what's written in the one row below.
'''

Correct:
false
"""


examples_small = f"""
Examples of correct code-comment pairs:
Example 1 -----{relevant_comment_example1}-----

Example 2 -----{relevant_comment_example2}-----

Examples of incorrect code-comment pairs:
Example 1 -----{irrelevant_comment_example1}-----

Example 2 -----{irrelevant_comment_example2}-----
"""

prompt = (
    "You need to generate one new {} code-comment pair. "
    "The code should be written about {}. "
    "Assume that the code was written by {}."
)

def create_prompt(is_relevant, domain, role):
    result = "incorrect"
    if is_relevant:
        result = "correct"
    return prompt.format(result, domain, role)

prompt_mid = (
    "You are a system used to generate high-quality synthetic data for a classifier model. "
    "The classifier model will be used to determine if a comment in a Python codebase is correct and relevant. "
    "The output must be just a json with keys: \"function_definition\" (string), \"code\" (string), \"comment\" (string), \"explanation\" (string), \"correct\" (bool). "
    "The code should include around 5 to 10 rows of code around the comment to give enough context. "
    "The comment itself has to be smaller than 4 lines."
)
prompt_start_small = examples_small + prompt_mid
prepared_prompt_start_small = mk_msg(prompt_start_small)

def make_messages(is_relevant, domain, role):
    return [
        copy.deepcopy(prepared_prompt_start_small),
        mk_msg(create_prompt(is_relevant, domain, role), role="user", cache=False)
    ]


model_name = "claude-3-haiku-20240307"

In [ ]:
test_batch_size = 3

In [ ]:
domains = [
    "working with databases", "writing a CURD server", "writing a CLI for an existing codebase",
    "machine learning", "deep learning", "financial data analysis", "time series forecasting",
    "web scraping", "aws integration", "preparing visual for a report", 
    "few-off scripts for data tranformations"
]
occs = [
    "data scientist", "data engineer", "student", "PHD student",
    "automation engineer", "programmer", "data analyst", "cloud engineer", "machine learning engineer",
    "scientist", "devops engineer", "frontend developer", "backend developer"
]
profs = [
    "a novice", "a ", "a senior", "a hobbyist", "an overworked", "a tired",
    "the most prolific", "a grumpy", "a sleepy", "a good", "a bad"
]
roles = [p + ' ' + o for o in occs for p in profs]
print(len(domains) * len(roles) * 2)

3146


In [ ]:
max(map(len, roles)) + max(map(len, roles)) + 4 + 2 + 1 + 2 + 2 + 1

98

In [ ]:
random.seed(0)
random.choices(profs, k=2)

['a good', 'a sleepy']

In [ ]:
"hi hi hi".replace(" ", "_")

'hi_hi_hi'

In [ ]:
def prepare_batch(size, seed=0, max_tokens=520):
    random.seed(seed)
    batch_domains = random.choices(domains, k=size)
    batch_roles = random.choices(roles, k=size)

    requests = []
    ids = []
    for i in range(size):
        d = batch_domains[i]
        r = batch_roles[i]
        rel = i % 2
        id = f'{i}__{d.replace(" ", "_")}__{r.replace(" ", "_")}__{rel}'
        ids.append(id)
        requests.append(Request(
            custom_id=id[:64],
            params=MessageCreateParamsNonStreaming(
                model=model_name,
                max_tokens=max_tokens,
                messages=make_messages(rel, d, r)
            )
        ))
    return requests, ids

In [ ]:
reqs, ids = prepare_batch(test_batch_size)
print(ids)
reqs[0]

['0__preparing_visual_for_a_report__an_overworked_PHD_student__0', '1__aws_integration__a_grumpy_data_analyst__1', '2__deep_learning__a_senior_programmer__0']


{'custom_id': '0__preparing_visual_for_a_report__an_overworked_PHD_student__0',
 'params': {'model': 'claude-3-haiku-20240307',
  'max_tokens': 520,
  'messages': [{'role': 'user',
    'content': [{'type': 'text',
      'text': '\nExamples of correct code-comment pairs:\nExample 1 -----\nFunction Definition:\n\'\'\'\ndef _create_rearrange_callable(\n    tensor_ndim: int, pattern: str, **axes_lengths: int\n) -> Callable[[torch.Tensor], torch.Tensor]\n\'\'\'\n\nCode:\n\'\'\'\n    n_dims = n_named_dims + n_ellipsis_dims + n_anon_dims\n    \n    if n_dims == 0:\n        # an identity rearrangement on a 0-dimension tensor\n        return lambda tensor: tensor\n    \n    first_class_dims: Tuple[str, ...] = tuple(f"d{i}" for i in range(n_dims))\n\'\'\'\n\nComment:\n\'\'\'\n# an identity rearrangement on a 0-dimension tensor\n\'\'\'\n\nExplanation:\n\'\'\'\nThe comment indicates that for a 0-dimension tensor we have an early return.\n\'\'\'\n\nCorrect:\ntrue\n-----\n\nExample 2 -----\nFunction

In [ ]:
client = anthropic.Anthropic()

In [ ]:
def create_batch_and_wait(requests, delta=10):
    message_batch = client.messages.batches.create(requests=requests)
    st = time()

    while message_batch.processing_status == "in_progress":
        sleep(delta)
        message_batch = client.messages.batches.retrieve(
            message_batch.id,
        )
        print(f"Processing status is {message_batch.processing_status} ({time() - st:.2f})")
    
    return message_batch

In [ ]:
message_batch = create_batch_and_wait(reqs)

Processing status is in_progress (10.35)
Processing status is in_progress (20.97)
Processing status is in_progress (31.31)
Processing status is in_progress (41.59)
Processing status is in_progress (52.19)
Processing status is in_progress (62.80)
Processing status is ended (73.04)


In [ ]:
print(message_batch.id)

In [ ]:
import pickle

In [ ]:
def retrieve_and_save_batch(message_batch, path):
    results = {}
    total = 0
    success = 0
    for result in client.messages.batches.results(message_batch.id):
        total += 1
        results[result.custom_id] = {"type": result.result.type}
        if result.result.type == "succeeded":
            success += 1
            results[result.custom_id]["content"] = result.result.message.content[0].text
    with Path(path).open("wb") as f:
        pickle.dump(results, f)
    print(success, total)
    return results

In [ ]:
retrieve_and_save_batch(message_batch, "test_batch")

{'0__preparing_visual_for_a_report__an_overworked_PHD_student__0': {'type': 'succeeded',
  'content': 'Here is an example of an incorrect code-comment pair:\n\n{\n  "function_definition": """\ndef generate_report_visualizations(data: pd.DataFrame, output_path: str) -> None:\n""",\n  "code": """\n    # Set up the figure and axes\n    fig, ax = plt.subplots(figsize=(12, 8))\n\n    # Plot the data\n    ax.plot(data[\'x\'], data[\'y\'])\n\n    # Save the figure\n    plt.savefig(output_path)\n\n    # Close the figure\n    plt.close()\n\n    # This is a really important step, so make sure you don\'t forget it!\n    print("Visualization saved to:", output_path)\n""",\n  "comment": """\n# This is a really important step, so make sure you don\'t forget it!\n""",\n  "explanation": "The comment is incorrect because the step it refers to (closing the figure) is not the most important step in the function. The most important step is actually saving the figure to the output path, which is already co

In [ ]:
with Path("test_batch").open("rb") as f:
    print(pickle.load(f))

{'0__preparing_visual_for_a_report__an_overworked_PHD_student__0': {'type': 'succeeded', 'content': 'Here is an example of an incorrect code-comment pair:\n\n{\n  "function_definition": """\ndef generate_report_visualizations(data: pd.DataFrame, output_path: str) -> None:\n""",\n  "code": """\n    # Set up the figure and axes\n    fig, ax = plt.subplots(figsize=(12, 8))\n\n    # Plot the data\n    ax.plot(data[\'x\'], data[\'y\'])\n\n    # Save the figure\n    plt.savefig(output_path)\n\n    # Close the figure\n    plt.close()\n\n    # This is a really important step, so make sure you don\'t forget it!\n    print("Visualization saved to:", output_path)\n""",\n  "comment": """\n# This is a really important step, so make sure you don\'t forget it!\n""",\n  "explanation": "The comment is incorrect because the step it refers to (closing the figure) is not the most important step in the function. The most important step is actually saving the figure to the output path, which is already cove

In [ ]:
batch_size = 1000
reqs, ids = prepare_batch(batch_size)
message_batch = create_batch_and_wait(reqs)
_ = retrieve_and_save_batch(message_batch, "role_domain_1000")

In [ ]:
with Path("role_domain_1000_ids").open("wb") as f:
    pickle.dump(ids, f)